In [59]:
import h3
import json

## Hexagon

In [60]:
lat = 10.7780202
lng = 106.7019901
resolution = 12  # 12 = 307.092 m2

# Generate a H3 index from a latitude and longitude pair:
h3_cell = h3.geo_to_h3(lat, lng, resolution)
print(h3_cell)

8c65b5662ab47ff


In [61]:
# Convert a H3 index back to latitude and longitude:
(lat, lng) = h3.h3_to_geo(h3_cell)
print(lat, lng)

10.777991032274201 106.70193163986322


## Children

In [62]:
# children of a H3 cell
children = h3.h3_to_children(h3_cell, resolution + 1)
print(type(children))

<class 'set'>


## Traversal

In [63]:
# distance
distance_km = 5

### Hex Range Distance

In [64]:
# hex_range_distance
rings_5km = h3.hex_range_distances(h3_cell, distance_km)

current_ring = 0
for ring in rings_5km:
    current_ring += 1
    print(f'Ring {current_ring}: {len(ring)} cells')
    
    for cell in ring:
        print(f'Cell: {cell}, Center: {h3.h3_to_geo(cell)}')
        
    print('-------------------')

Ring 1: 1 cells
Cell: 8c65b5662ab47ff, Center: (10.777991032274201, 106.70193163986322)
-------------------
Ring 2: 6 cells
Cell: 8c65b5662ab41ff, Center: (10.777823581056142, 106.70200578431663)
Cell: 8c65b5662ab0dff, Center: (10.778158483540526, 106.70185749529813)
Cell: 8c65b5662ab45ff, Center: (10.777971106444836, 106.7021137785153)
Cell: 8c65b5662ab09ff, Center: (10.778010958019848, 106.70174950097996)
Cell: 8c65b5662ab43ff, Center: (10.777843506745985, 106.70182364561758)
Cell: 8c65b5662ab6bff, Center: (10.778138557655359, 106.70203963413444)
-------------------
Ring 3: 12 cells
Cell: 8c65b5662ab4dff, Center: (10.777803655282586, 106.70218792278449)
Cell: 8c65b56621693ff, Center: (10.777951180531762, 106.7022959169362)
Cell: 8c65b5662ab49ff, Center: (10.777656129886356, 106.70207992865835)
Cell: 8c65b5662ab05ff, Center: (10.778325934855118, 106.70178335062138)
Cell: 8c65b5662ab55ff, Center: (10.777863432352115, 106.70164150668737)
Cell: 8c65b5662ab4bff, Center: (10.77767605552039

### K-ring

In [65]:
# K-ring of a H3 cell:
k_ring_5km = h3.k_ring(h3_cell, distance_km)

print(type(k_ring_5km))
for cell in k_ring_5km:
    print(f'Cell: {cell}, Center: {h3.h3_to_geo(cell)}')

<class 'set'>
Cell: 8c65b5662b9a1ff, Center: (10.777468753102909, 106.70233621098781)
Cell: 8c65b5662ab4bff, Center: (10.777676055520391, 106.70189779014353)
Cell: 8c65b5662b987ff, Center: (10.777173702133263, 106.70212022305132)
Cell: 8c65b566216b1ff, Center: (10.778393755396916, 106.70261989954483)
Cell: 8c65b5662ab6dff, Center: (10.778266156781049, 106.70232976698942)
Cell: 8c65b566216d7ff, Center: (10.777743877459665, 106.7027343368009)
Cell: 8c65b5662ab17ff, Center: (10.778238186244042, 106.70112893764112)
Cell: 8c65b56621693ff, Center: (10.777951180531762, 106.7022959169362)
Cell: 8c65b5662ab05ff, Center: (10.778325934855118, 106.70178335062138)
Cell: 8c65b5662ab25ff, Center: (10.778788436787964, 106.70192519483626)
Cell: 8c65b5662ab5dff, Center: (10.777695981070716, 106.70171565139754)
Cell: 8c65b5662ab65ff, Center: (10.77843360797649, 106.70225562275365)
Cell: 8c65b5662ab43ff, Center: (10.777843506745985, 106.70182364561758)
Cell: 8c65b5662ab57ff, Center: (10.777883357874535, 1

## H3 to GeoJSON

In [67]:
def h3_to_geojson(h3_cells: set) -> str:
    # Iterate H3 indexes and Transform into GeoJSON:
    features = []
    for cell in h3_cells:
        polygon_geometry = {
            "type": "Polygon",
            "coordinates": [list(h3.h3_to_geo_boundary(h=cell, geo_json=True))],
        }

        feature = {
            "type": "Feature",
            "geometry": polygon_geometry,
            "properties": {
                "h3_idx": cell,
                "is_valid_h3_cell": h3.h3_is_valid(cell),
                "center": h3.h3_to_geo(cell),
                "area_m2": h3.cell_area(h=cell, unit="m^2"),
                "resolution": h3.h3_get_resolution(cell)
            }
        }

        features.append(feature)

    # Wrap Features into a FeatureCollection:
    geojson_FeatureCollection = {
        "type": "FeatureCollection", 
        "features": features
    }
    
    # Serialize to JSON:
    geojson = json.dumps(geojson_FeatureCollection)
    
    return geojson

geojson = h3_to_geojson(k_ring_5km)
print(geojson)

{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[106.70242163827288, 10.777406294192028], [106.70243292163038, 10.777511286285916], [106.70234749432569, 10.77757374522303], [106.70225078362368, 10.777531212022259], [106.70223950031195, 10.777426219863257], [106.70232492765648, 10.77736376097014], [106.70242163827288, 10.777406294192028]]]}, "properties": {"h3_idx": "8c65b5662b9a1ff", "is_valid_h3_cell": true, "center": [10.777468753102909, 106.70233621098781], "area_m2": 352.50613383487814, "resolution": 12}}, {"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[106.70198321780418, 10.77761359659341], [106.70199450105713, 10.7777185888311], [106.70190907337687, 10.77778104778431], [106.70181236240384, 10.777738514455837], [106.70180107919666, 10.777633522153037], [106.70188650691676, 10.777571063243823], [106.70198321780418, 10.77761359659341]]]}, "properties": {"h3_idx": "8c65b5662ab4bff", "is_valid_h3_ce